In [1]:
import pandas as pd
import numpy as np
import re
import nltk

dataset = pd.read_csv("YoutubeCommentsDataSet.csv")

df = dataset.copy()
sentiment_map = {'negative': 0,'neutral': 1,'positive': 2}

df['sentiment_encoded'] = df['Sentiment'].map(sentiment_map)



print(df)
df2 = df.iloc[:,2:]
print(df2)



                                                 Comment Sentiment  \
0      lets not forget that apple pay in 2014 require...   neutral   
1      here in nz 50 of retailers don’t even have con...  negative   
2      i will forever acknowledge this channel with t...  positive   
3      whenever i go to a place that doesn’t take app...  negative   
4      apple pay is so convenient secure and easy to ...  positive   
...                                                  ...       ...   
18403  i really like the point about engineering tool...  positive   
18404  i’ve just started exploring this field and thi...  positive   
18405  excelente video con una pregunta filosófica pr...   neutral   
18406  hey daniel just discovered your channel a coup...  positive   
18407  this is great focus is key a playful approach ...  positive   

       sentiment_encoded  
0                      1  
1                      0  
2                      2  
3                      0  
4                      2

In [2]:
comment = df.iloc[:,:1]
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alper\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer  # or TfidfVectorizer

nltk.download('stopwords')
ps = PorterStemmer()

corpus = []
new_labels = []  # to collect matching y values

for i in range(len(comment)):
    text = comment.iloc[i, 0]
    if pd.isna(text):  # skip empty rows
        continue
    review = re.sub('[^a-zA-Z]', ' ', str(text))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    if review.strip() != '':
        corpus.append(review)
        new_labels.append(df['sentiment_encoded'].iloc[i])

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000)
X = cv.fit_transform(corpus).toarray()
y = np.array(new_labels)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alper\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Make predictions
y_pred = mnb.predict(X_test)
# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Accuracy: 0.714167585446527

Confusion Matrix:
[[ 261   28  178]
 [ 178  293  414]
 [ 118  121 2037]]


In [6]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print("LogReg Accuracy:", accuracy_score(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred))

LogReg Accuracy: 0.7378721058434399
[[ 261   28  178]
 [ 178  293  414]
 [ 118  121 2037]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred))

Random Forest Accuracy: 0.7331863285556781
[[ 261   28  178]
 [ 178  293  414]
 [ 118  121 2037]]
